## Online Nonnegative Matrix Factorization with General Divergences

In [52]:
using LinearAlgebra, Distributions, ForwardDiff, PyPlot, DataFrames,Divergences, SparseArrays, DelimitedFiles

In [53]:
function Π₁(v)
    w = v
    for i=1:length(v)
        if v[i]<0
            w[i] = 0.001
                           
        end
    end

   if norm(w,2)>1
        return (w/norm(w,2))
    end
            
    return w
end

Π₁ (generic function with 1 method)

#### Projection based on: Efficient Projections onto the ℓ1-Ball for Learning in High Dimensions

In [54]:
function Πᵪ(X) # fazendo z = 1
    m,n = size(X)
    z = 1
    
    W= Array{Float64,2}(undef, m, n)
    
    μ = [] # store the nonzeros values
    ρ = Inf
    for i=1:m
        μ = sort(X[i,:], rev=true)
        ρ= maximum([j for j=1:n if (μ[j]-((1/j)*sum([μ[r]-z for r=1:j ]))) > 0])
        θ = (1/ρ)*sum([(μ[i]-z) for i=1:ρ ])
        W[i,:] = [max(X[i,j] - θ, 0) for j=1:n]
    end
    
    
    return W
end

Πᵪ (generic function with 1 method)

## Learning $h^t$
$$h_k^t = \Pi_\mathcal{H} \Big\{ h_{k-1}^t - \beta_k^t g_k^t\Big \},\mathcal{H} \in \mathrm{R}^k_+$$


In [55]:
function learning_hₜ(W, vₜ, h₀, δ, βₜ)
    k = 0      
    d = 1
    hₜ = h₀  
    
    while d>δ      
       
        f(x) = evaluate(KullbackLeibler(), vₜ,(W*x).+(0.001))
        grad = gradient(KullbackLeibler(),  vₜ, (W*h₀).+(0.001))
        g = rand(1:length(grad))    
        hₜ = Π₁(h₀.-(βₜ*grad[g]) )      
        d = f(h₀)- f(hₜ)
        h₀= hₜ       
        k+=1      
       
    end
    return hₜ
end    

learning_hₜ (generic function with 1 method)

## Projection $W_t$
$$\Pi_\mathcal{C} \Big\{ W_{t-1} - \eta_t G_t\Big \},\mathcal{C} \in \mathrm{R}^{m,k}_+$$

In [56]:
function OnlineMNF(T, A, r, δ)    
    m = size(A)[1]    
    W = Array{Float64, 3}(undef, T, m,r)
    W_ = Array{Float64, 2}(undef, m,r)
    H = Array{Float64, 2}(undef,r, T)
    
    for i=1:m, j=1:r
        W_[i,j] = ceil(rand(0.0:1.0))
    end
    W_ =  W_/norm(W_, 1)
    
    h = [rand(Uniform(0,100)) for i=1:r ]
    h= h/norm(h,1) 
    
    v = (A[:,1].+(0.001))/norm((A[:,1].+(0.001)),1)
    βₜ = 1
     h = learning_hₜ(W_, v,h,δ, βₜ )
   
   
     
    grad = gradient(KullbackLeibler(), v, (W_*h).+(0.001))
    g = rand(1:length(grad)) 
    W[1,:,:] = Πᵪ( ( W_ .- grad[g])) 
    H[:,1] =  h
    L = 1
    
    
    for t=2:T
        @show t
        vₜ = (A[:,t].+(0.001))/norm((A[:,t].+(0.001)),1)
        h = [rand(Uniform(0,1)) for i=1:r ]
        h= h/norm(h,1) 
        
        hₜ = learning_hₜ(W[t-1,:,:], vₜ,h,δ, βₜ)
        f(h) = evaluate(KullbackLeibler(), vₜ, ((W[t-1,:,:]*h).+(0.001)))
        abs(f(H[:,t-1]) - f( hₜ))/norm(H[:,t-1] -  hₜ)
       
        
        grad = gradient(KullbackLeibler(), vₜ,(W[t-1,:,:]* hₜ).+(0.001)) 
        d = norm(grad)
        if L<d
            L = d
        end
      
         βₜ = 1/L
        g = rand(1:length(grad))       
        W[t,:,:] = Πᵪ( (W[t-1,:,:] .- (1/t)*grad[g]))       
        H[:,t] =  hₜ
        
    end
    return W, H
end
    

OnlineMNF (generic function with 1 method)

In [57]:
A = readdlm("test.txt")

28163×2157 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [58]:
X = sparse(A)

28163×2157 SparseMatrixCSC{Float64,Int64} with 213933 stored entries:
  [733  ,    1]  =  1.0
  [901  ,    1]  =  1.0
  [916  ,    1]  =  3.0
  [1209 ,    1]  =  1.0
  [1214 ,    1]  =  2.0
  [1415 ,    1]  =  1.0
  [1591 ,    1]  =  1.0
  [1742 ,    1]  =  1.0
  [2029 ,    1]  =  1.0
  [2527 ,    1]  =  1.0
  [2541 ,    1]  =  2.0
  [3178 ,    1]  =  1.0
  ⋮
  [19478, 2157]  =  1.0
  [19547, 2157]  =  1.0
  [20652, 2157]  =  1.0
  [23724, 2157]  =  2.0
  [23757, 2157]  =  1.0
  [23803, 2157]  =  1.0
  [24016, 2157]  =  1.0
  [24185, 2157]  =  1.0
  [24584, 2157]  =  1.0
  [26525, 2157]  =  1.0
  [26807, 2157]  =  2.0
  [27180, 2157]  =  1.0
  [27859, 2157]  =  2.0

In [86]:
samples = Array{Float64, 2}(undef,100 , size(A)[2])
index = Int64[]
i = 1
while i<=100
    j = rand(1:size(A)[2])
    if !(j in index)
        samples[:,i] = A[1:100,j]
        push!(index, j)
        i+=1
    end
end
X = sparse(samples)
""
    

""

In [64]:
@time W,H = OnlineMNF(size(X)[2], X, 4, 0.001)   

t = 2
t = 3
t = 4
t = 5
t = 6
t = 7
t = 8
t = 9
t = 10
t = 11
t = 12
t = 13
t = 14
t = 15
t = 16
t = 17
t = 18
t = 19
t = 20
t = 21
t = 22
t = 23
t = 24
t = 25
t = 26
t = 27
t = 28
t = 29
t = 30
t = 31
t = 32
t = 33
t = 34
t = 35
t = 36
t = 37
t = 38
t = 39
t = 40
t = 41
t = 42
t = 43
t = 44
t = 45
t = 46
t = 47
t = 48
t = 49
t = 50
t = 51
t = 52
t = 53
t = 54
t = 55
t = 56
t = 57
t = 58
t = 59
t = 60
t = 61
t = 62
t = 63
t = 64
t = 65
t = 66
t = 67
t = 68
t = 69
t = 70
t = 71
t = 72
t = 73
t = 74
t = 75
t = 76
t = 77
t = 78
t = 79
t = 80
t = 81
t = 82
t = 83
t = 84
t = 85
t = 86
t = 87
t = 88
t = 89
t = 90
t = 91
t = 92
t = 93
t = 94
t = 95
t = 96
t = 97
t = 98
t = 99
t = 100
t = 101
t = 102
t = 103
t = 104
t = 105
t = 106
t = 107
t = 108
t = 109
t = 110
t = 111
t = 112
t = 113
t = 114
t = 115
t = 116
t = 117
t = 118
t = 119
t = 120
t = 121
t = 122
t = 123
t = 124
t = 125
t = 126
t = 127
t = 128
t = 129
t = 130
t = 131
t = 132
t = 133
t = 134
t = 135
t = 136
t = 137
t = 138
t = 139
t 

([1.0027624309392267 1.0 … 1.0013812154696136 1.0013812154696136; 1.0027624309392267 1.0 … 1.0013812154696131 1.0013812154696131; … ; 1.0027624309392262 1.0 … 1.0013812154696133 1.0013812154696133; 1.0027624309392262 1.0 … 1.0013812154696133 1.0013812154696133]

[1.0027624309392267 1.0 … 1.0013812154696136 1.0013812154696136; 1.0027624309392267 1.0 … 1.0013812154696131 1.0013812154696131; … ; 1.0027624309392262 1.0 … 1.0013812154696133 1.0013812154696133; 1.0027624309392262 1.0 … 1.0013812154696133 1.0013812154696133]

[0.9972375690607738 1.0 … 0.9958563535911606 1.0013812154696136; 0.9972375690607738 1.0 … 0.9958563535911602 1.0013812154696131; … ; 0.9972375690607738 1.0 … 0.9958563535911598 1.0013812154696133; 0.9972375690607738 1.0 … 0.9958563535911598 1.0013812154696133]

[0.9972375690607738 1.0 … 1.0013812154696136 0.9958563535911606; 0.9972375690607738 1.0 … 1.0013812154696131 0.9958563535911602; … ; 0.9972375690607738 1.0 … 1.0013812154696133 0.9958563535911598; 0.99723756906077

In [83]:
function clustering_NMF(H)
    l,c = size(H)
    labels_NMF = []
   
    for index =1:c

        col = H[:,index]

        cluster = argmax(col)
        

        push!(labels_NMF,cluster)
    end
    
    return labels_NMF
 end

clustering_NMF (generic function with 1 method)

In [84]:
a = clustering_NMF(H)

2157-element Array{Any,1}:
 3
 1
 3
 1
 2
 4
 1
 4
 1
 1
 2
 3
 3
 ⋮
 1
 1
 3
 2
 2
 2
 3
 4
 2
 4
 2
 1